In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
  !pip install tensorflow-datasets
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

## Data Preprocessing

This code will load the dataset, display basic information about its structure, show the first few rows of the training data, provide statistics about both datasets, show the distribution of labels, and display sample messages for both ham and spam categories.

In [ ]:
# Load the dataset
train_data = pd.read_csv(train_file_path, sep='\t', names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', names=['label', 'message'])

# Display basic information about the datasets
print("Training data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

# Display the first few rows of the training data
print("\nFirst few rows of training data:")
print(train_data.head())

# Display basic statistics of the datasets
print("\nTraining data info:")
train_data.info()

print("\nTest data info:")
test_data.info()

# Display the distribution of labels in the training set
print("\nLabel distribution in training set:")
print(train_data['label'].value_counts(normalize=True))

# Display some sample messages
print("\nSample ham message:")
print(train_data[train_data['label'] == 'ham']['message'].iloc[0])

print("\nSample spam message:")
print(train_data[train_data['label'] == 'spam']['message'].iloc[0])

In [ ]:
import re
from nltk.tokenize import word_tokenize

# Download NLTK punkt for tokenization
import nltk
nltk.download('punkt')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Join tokens back into a string
    processed_text = ' '.join(tokens)

    return processed_text

# Apply preprocessing to both training and test data
train_data['processed_message'] = train_data['message'].apply(preprocess_text)
test_data['processed_message'] = test_data['message'].apply(preprocess_text)

# Display a sample of preprocessed messages
print("Sample preprocessed ham message:")
print(train_data[train_data['label'] == 'ham']['processed_message'].iloc[0])

print("\nSample preprocessed spam message:")
print(train_data[train_data['label'] == 'spam']['processed_message'].iloc[0])



This code accomplishes the following:

1. Creates a vocabulary using Keras' Tokenizer
2. Converts the preprocessed messages to sequences of word indices
3. Pads the sequences to ensure uniform length
4. Converts the labels to numerical values (0 for ham, 1 for spam)
With these steps completed, our data is now ready for model creation and training. The next part will involve building and training the neural network model.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create a vocabulary of unique words
max_vocab_size = 10000  # You can adjust this value
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['processed_message'])

# Convert messages to sequences of word indices
train_sequences = tokenizer.texts_to_sequences(train_data['processed_message'])
test_sequences = tokenizer.texts_to_sequences(test_data['processed_message'])

# Pad sequences to ensure uniform length
max_length = 100  # You can adjust this value
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

# Convert labels to numerical values
train_labels = (train_data['label'] == 'spam').astype(int)
test_labels = (test_data['label'] == 'spam').astype(int)

print("Vocabulary size:", len(tokenizer.word_index))
print("Shape of training data:", train_padded.shape)
print("Shape of test data:", test_padded.shape)


## 2. Model Creation

This code creates a neural network with the following architecture:

An Embedding layer to learn word representations
1. Two LSTM layers for sequence processing
2. A Dense layer with ReLU activation
3. A Dropout layer for regularization
4. A final Dense layer with sigmoid activation for binary classification

The model is then compiled using the Adam optimizer and binary cross-entropy loss function. We also include accuracy as a metric to monitor during training.

This structure follows the approach outlined in the Approach.md file and should provide a good starting point for the SMS spam classification task. You can adjust the hyperparameters (such as embedding_dim, lstm_units, and the number of layers) to experiment with different model configurations.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16
lstm_units = 32

# Build the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


# Print model summary
model.summary()


## 3 Model Training

This code implements all five steps mentioned in the Model Training section:

1. Splits the data into training and validation sets.
2. Uses the already separated features (X) and labels (y).
3. Trains the model using the preprocessed data.
4. Monitors validation accuracy by plotting training and validation metrics.
5. Provides a commented example of how to adjust hyperparameters.

_Key Point:_ The class_weighting means that during training, the model gives twice as much importance to the spam class (labeled as 1) compared to the ham class (labeled as 0). This weighting scheme is particularly useful when dealing with imbalanced datasets, where spam messages might be less frequent than ham messages.

After running this code, you'll have a trained model and visualizations of its performance over epochs. You can then analyze these results to determine if further hyperparameter tuning is necessary.

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_padded, train_labels, test_size=0.2, random_state=42)
print(np.unique(y_train))
print(np.unique(y_val))

# Convert to numpy arrays
x_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_val_np = np.array(X_val)
y_val_np = np.array(y_val)

# Step 2: Features and labels are already separated in previous steps

# Step 3: Train the model using the preprocessed data
# Train the model with d class weights
class_weights = {0: 1.0, 1: 2.0}  # 0 for ham, 1 for spam
history = model.fit(
    x=x_train_np,
    y=y_train_np,
    epochs=10,
    batch_size=64,
    validation_data=(X_val_np, y_val_np),
    class_weight=class_weights,
    verbose=1
)


# Step 4: Monitor validation accuracy

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Step 5: Adjust hyperparameters as needed
# You can experiment with different values for epochs, batch_size, and learning rate
# For example:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_val, y_val), verbose=1)


## 4. Implement predict_message Function

This implementation follows the three steps outlined in the Approach.md file:

1. It preprocesses the input message using the same preprocess_text function used for training data.
2. It uses the trained model to make a prediction on the preprocessed message.
3. It returns the probability and corresponding label (ham or spam) based on the prediction.

The function is then tested with a few example messages to demonstrate its functionality. You can run this code to see how the model performs on these examples and add more test cases as needed.

In [ ]:
def predict_message(message):
    # Step 1: Preprocess the input message
    processed_message = preprocess_text(message)

    # Step 2: Use the trained model to make a prediction
    sequence = tokenizer.texts_to_sequences([processed_message])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded_sequence)[0][0]

    # Step 3: Return the probability and corresponding label
    label = "spam" if prediction > 0.5 else "ham"
    return [float(prediction), label]

# Test the function with a few examples
examples = [
    "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
    "Hey, how are you doing? Want to meet up later?",
    "WINNER!! As a valued network customer you have been selected to receive a £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.",
    "Don't forget we have a meeting at 3pm today."
]

for example in examples:
    prediction = predict_message(example)
    print(f"Message: {example}")
    print(f"Prediction: {prediction}")
    print()


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(f"Expected value: {ans}, but got {prediction[1]}")
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
